# Circuit-QED

This notebook contains some useful models for superconducting circuit setups, such as charge qubits, flux qubits and combinations thereof.

In [ ]:
# file: seeq/models/cqed.py
import numpy as np
import scipy.sparse as sp
from scipy.sparse.linalg import LinearOperator
from seeq.tools import lowest_eigenstates, lowest_eigenvalues

## Transmon qubit

### Effective model

A charge qubit is a superconductiong circuit made of a capacitor and a Josephson junction (See Fig. a) below). The transmon is a variant of the charge qubit where the Josephson energy is much larger than the capacitive energy. This change makes the transmon less sensitive to charge and voltage fluctuations.

![Charge and transmon qubits, and equivalent circuit](figures/transmon.png)

Following the image above, the Hamiltonian for the transmon qubit is
$$\hat{H}=\frac{1}{2C_\Sigma} (\hat{q}-q_g)^2-E_J \cos(\hat{\varphi})$$
where $q_g=-C_g V$ is the equilibrium charge, $C_g$ is the control capacitance, $V$ the electric potential of the battery, $E_J$ the Josephson junction energy, and $C_\Sigma$ the combined qubit and charging capacitance. $\hat{q}$ is the charge operator of the Cooper pairs, and $\hat{\varphi}$ the flux operator.

We work with this Hamiltonian by introducing the charge number states
$$\hat{q} \lvert n \rangle = -2e n \lvert n \rangle, \quad \text{and} \quad \langle \varphi \lvert n \rangle \sim \frac{1}{\sqrt{2\pi}} e^{-in\varphi}$$

Using the number basis we can represent $\cos(\hat{\varphi})$ as
$$\cos(\hat{\varphi})=\frac{1}{2}\sum_{n\in\mathbb{Z}} \lvert n+1\rangle \langle n\rvert + \vert n \rangle \langle n+1 \rvert$$

Then we can express the charge qubit Hamiltonian in the number representation as
$$ H=\sum_{n\in\mathbb{Z}} \left[ 4 E_C (n-n_g)^2 \lvert n\rangle \langle n\rvert - \frac{E_J}{2}(\lvert n+1\rangle \langle n\rvert + \vert n \rangle \langle n+1 \rvert)\right]$$
with the charging energy $E_C=e^2/2C_\Sigma$.

This operator is an infinite-dimensional matrix
$$H = \left(\begin{matrix}
\ldots & -E_J/2 & 4E_c(-1-n_g)^2 & -E_J/2 & 0 & \ldots\\
\ldots & 0 & -E_J/2 & 4E_c(0-n_g)^2 & -E_J/2 & \ldots\\
\ldots & 0 & 0 & -E_J/2 & 4E_c(+1-n_g)^2 & \ldots
\end{matrix}\right)$$
but it can be truncated to a work in a limit $-n_{max}\leq n \leq n_{max},$ considering enough charge states as to provide a good approximation. In this case, we can write the model as
$$H = 4 E_C (\bar{N}-n_g)^2 - \frac{1}{2}E_J (\bar{S}^+ + \bar{S}^-),$$
with finite dimensional matrices $\bar{N}$ for the number of charges, and charge raising and lowering operators $\bar{S}^\pm.$

We can also introduce a model in which we have $M$ transmons interacting with each other capacitively. If we neglect the renormalization of the transmon capacitance, the effective model reads
$$H = \sum_i \left[4 E_{C,i} (\bar{N}_i-n_{g,i})^2 + \frac{1}{2}E_{J,i}(\bar{S}^+_i + \bar{S}^-_i)\right] + \sum_{i\neq j} g_{ij} \bar{N}_i\bar{N}_j.$$

In [ ]:
# file: seeq/models/cqed.py

class Transmons(LinearOperator):
    
    """Transmons() implements one or more coupled transmons. This class
    acts as a LinearOperator that implements the Hamiltonian. It can
    also produce copies of itself with adjusted parameters. If a parameter
    is a scalar, the same value is used for all qubits.
    
    Parameters
    ----------
    nqubits -- number of transmons
    Ec      -- capacitive energy (defaults to 1/95.)
    EJ      -- Josephson energy (defaults to 1.).
    g       -- couplings (scalar or matrix)
    ng      -- offset in number (defaults to 0)
    nmax    -- cutoff in charge space (defaults to 8)
    format  -- format of matrices (defaults to 'csr')
    """

    def __init__(self, nqubits, Ec=1/95., EJ=1., g=0, ng=0, nmax=8, format='csr'):
        self.nqubits = nqubits

        # Dimensions of one-qubit problem
        dim = 2*nmax+1
        # Dimension of operators and states for the full problem
        fulldim = dim**nqubits
        #
        # This class inherits from LinearOperator because that implements
        # useful multiplication operators.
        super(Transmons,self).__init__(np.float64, (fulldim,fulldim))       
        #
        # Operators for one qubit
        self.nmax = nmax
        N = sp.diags(np.arange(-nmax, nmax + 1, 1), 0,
                     shape=(dim, dim), format=format)
        Sup = sp.diags([1.0], [1], shape=(dim,dim), format=format)
        Sdo = Sup.T
        #
        # Extend an operator to act on the whole Hilbert space
        def qubit_operator(op, j, N):
            d = op.shape[0]
            il = sp.eye(d**j, format=format)
            ir = sp.eye(d**(N-j-1), format=format)
            return sp.kron(il, sp.kron(op, ir))
        #
        # Local operators on all qubits:
        #
        self.N = [qubit_operator(N, j, nqubits) for j in range(nqubits)]
        self.nmax = nmax
        #
        # Capacitive energy
        self.Ec = Ec = Ec * np.ones(nqubits)
        self.ng = ng = ng * np.ones(nqubits)
        Id = sp.eye(fulldim)
        self.Hcap = sum((4.0*Ec) * (N-ng*Id)**2
                        for ng, Ec, N in zip(ng, self.Ec, self.N))
        #
        # Inductive energy
        self.EJ = EJ = EJ * np.ones(nqubits)
        self.HJJ = [EJ * qubit_operator((Sup+Sdo)/2., j, nqubits)
                     for j, EJ in enumerate(self.EJ)]
        #
        # The interaction must be symmetric
        g = g * np.ones((nqubits, nqubits))
        self.g = g = (g + g.T)/2.0
        self.Hint = sum((2.0 * g[i,j]) * (self.N[i] * self.N[j])
                         for i in range(self.nqubits)
                         for j in range(i)
                         if g[i,j])

    def _normalize_EJ(self, EJ):
        return self.EJ if EJ is None else EJ * np.ones(self.nqubits)

    def hamiltonian(self, EJ=None, gfactor=1.):
        """Return the Hamiltonian of this set of transmons, possibly
        changing the Josephson energies or rescaling the couplings.
        
        Arguments:
        ----------
        EJ      -- A scalar or a vector of Josephson energies, or
                   None if we use the default values.
        gfactor -- Multiplicative factor on the interaction term.
        """
        EJ = self._normalize_EJ(EJ)
        return sum((-EJ) * hi for EJ, hi in zip(EJ,self.HJJ)) + \
                self.Hcap + gfactor * self.Hint

    def apply(self, ψ, EJ=None, gfactor=1.):
        """Act with the Hamiltonian of this set of transmons, onto
        the state ψ. Arguments are the same as for hamiltonian().
        """
        EJ = self._normalize_EJ(EJ)
        out = self.Hcap @ ψ - sum(EJi * (hi @ ψ) for EJi, hi in zip(EJ,self.HJJ))
        if gfactor and self.Hint is not 0:
            out += gfactor * (self.Hint @ ψ)
        return out

    def _matvec(self, A):
        return self.apply(A)

    def qubit_basis(self, EJ=None, which=None):
        """Return the computational basis for the transmons in the limit
        of no coupling.
        
        Arguments:
        ----------
        which -- If None, return all 2**nqubits eigenstates. If it is
                 an index, return the eigenstates for the n-th qubit.
        EJ    -- Josephson energy (or None, for the default values)
        
        Returns:
        --------
        ψ     -- Matrix with columns for the computational basis states.
        """
        nqubits = self.nqubits
        EJ = self._normalize_EJ(EJ)
        if which is None:
            basis = 1
            for i in range(nqubits):
                basis = np.kron(basis, self.qubit_basis(ϵ, i))
        else:
            ti = Transmons(nqubits=1, Ec=self.Ec[i], nmax=self.nmax)
            _, basis = ti.eigenstates(2, EJ=self.EJ[i])
        return basis

We can plot the change of the spectrum as a function of the external potential. Notice how the sensitivity to the external field $n_g$ vanishes as we increase the ratio $E_J/E_c.$

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

def test():
    ng = np.linspace(-1., 1., 21)
    fig, (ax1, ax2, ax3) = plt.subplots(ncols=3, figsize=(10,3))
    for (Ec, s, axi) in [(1.0, 'b--', ax1),
                         (1/10., 'k-', ax2),
                         (1/40., 'g-.', ax3)]:
        λ = np.array([lowest_eigenvalues(Transmons(1, Ec, ng=n, EJ=1.), 3)
                      for n in ng])
        axi.plot(ng, λ[:,0], s, label=f'Ec={Ec}')
        axi.plot(ng, λ[:,1:], s)
        axi.set_xlabel('$n_g$')
        axi.set_ylabel('$E/E_J$')
        axi.set_title(f'$E_C={Ec}E_J$')
    plt.show()
    
test()